<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/non_Markov_6_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install japanize_matplotlib
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 55.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120258 sha256=b5c5621f5c3f05810a2a414a399bb37fc375a0a3c5793f3ba44f2b48e3662667
  Stored in directory: /root/.cache/pip/wheels/8c/6b/86/8d53dd07a93ebf907e5ba60f380ee2f932880c25f7a55026e4
Successfully built japanize_matplotlib


In [ ]:
#@title 関数 ストラングル 3

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.stats import norm
import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 5# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算



def BS_price( S, K, r, sigma, T, payoff):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif payoff == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError('payoff must be either "call" or "put"')
#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める




#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ



#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    payoff='bull'
    # 支払い条件に応じたb_n, c_nの計算
    if payoff == 'call':
        b_n = torch.clamp(stock_price - strike, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'strangle':
        b_n = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'bull':
        b_n = torch.clamp(stock_price - 80, min=0).cuda() - torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'batafrei':
        b_n = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'strangle_batafrei':
        strangle = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        batafrei = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        b_n = strangle + batafrei
        c_n = b_n ** 2
    elif payoff == 'put_bull':
        put = torch.clamp(80 - stock_price, min=0).cuda()
        bull = torch.clamp(stock_price - 80, min=0).cuda() - torch.clamp(stock_price - 120, min=0).cuda()
        b_n = put + bull
        c_n = b_n ** 2
    elif payoff == 'strangle_2batafrei':
        strangle = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        batafrei = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        b_n = strangle + 2 * batafrei
        c_n = b_n ** 2
    elif payoff == 'M_type':
        put=torch.clamp(80-stock_price, min=0).cuda()
        batafrei=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
        call=torch.clamp(stock_price-120, min=0).cuda()
        b_n=-put-2*batafrei-call+40
        c_n=b_n**2

    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0

In [ ]:
Call_T

array([[26.1553924 , 20.22449372],
       [21.52870986, 19.10092681],
       [19.17347615, 19.0384264 ],
       [20.01343232, 18.19847024],
       [31.3017807 ,  6.91012185],
       [14.93126622,  0.        ],
       [14.93126622,  0.        ],
       [ 0.        ,  0.        ],
       [50.30437758, 14.93126622],
       [14.93126622,  0.        ],
       [14.93126622,  0.        ],
       [ 0.        ,  0.        ],
       [14.93126622,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [ ]:
dt

0.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

param_to_prob(mu=0.03, sigma=0.25,nu=nu,zeta=zeta,dt=dt)

array([0.25110906, 0.48979592, 0.25909502])

In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ストラングル 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =np.arange(141,150,1)
strike_range=[200]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #init_cost= BS_price(100,strike,0,0.3,1,'call')
    #init_cost= BS_price(100,80,0,0.3,1,'call')+BS_price(100,120,0,0.3,1,'call')-20
    #init_cost =BS_price(100,80,0,0.3,1,'call')-BS_price(100,120,0,0.3,1,'call')
    #init_cost =BS_price(100,80,0,0.3,1,'call')-2*BS_price(100,100,0,0.3,1,'call')+BS_price(100,120,0,0.3,1,'call')
    def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
      stock_price=np.array(S_0)
      for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

      return stock_price
    payoff='bull'
    if payoff == 'call':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    elif payoff == 'strangle':
      Call_T=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'bull':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'batafrei':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'strangle_batafrei':
      strangle=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=strangle+batafrei
    elif payoff == 'put_bull':
      put=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)
      bull=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=put+bull
    elif payoff == 'strangle_2batafrei':
      strangle=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=strangle+2*batafrei
    elif payoff == 'M_type':
      put=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      call=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=-put-2*batafrei-call+40
    risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める


    for n in np.arange(maturity-1,-1,-1):
      Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    print('初期コスト',init_cost)
    def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
      stock_price=np.array(S_0)
      for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

      return stock_price

    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)


    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 2**6
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5*2

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        if epoch % 1000==0:
            print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

初期コスト 18.387517628638843
0 4.429898471811612
1000 10.362193751159168
2000 10.389088484416845
3000 10.389239756284553
4000 10.391903076040649
5000 10.392564980992518
6000 10.379279207568175
7000 10.437886214032517
8000 10.437892121408538
9000 10.437841095123702
10000 10.437900153713883
11000 10.437901032415596
12000 10.437902266285903
13000 10.437901894645222
14000 10.437904129630397
15000 10.437905668724795
16000 10.437899332385712
17000 10.4378967900941
18000 10.437905982958853
19000 10.43789026909559
20000 10.437868962451432
21000 10.437906094691925
22000 10.43790613626976
23000 10.43790593344221
24000 10.437906325363542
25000 10.429837459801377
26000 10.429825112623973
27000 10.429837474118926
28000 10.42982536043445
29000 10.429837475535294
30000 10.42983745497952
31000 10.429837469292522
32000 10.429837473223472
33000 10.429837419663954
34000 10.429837472600582
35000 10.429837474997157
36000 10.429836800389069
37000 10.43118807005203
38000 10.4311888013994
39000 10.431188798221001

In [ ]:
init_cost

13.173790908675008

In [ ]:
#@title 関数 ストラングル 3

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.stats import norm
import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 5# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算



def BS_price( S, K, r, sigma, T, payoff):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif payoff == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError('payoff must be either "call" or "put"')
#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める




#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ



#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    payoff='put_bull'
    # 支払い条件に応じたb_n, c_nの計算
    if payoff == 'call':
        b_n = torch.clamp(stock_price - strike, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'strangle':
        b_n = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'bull':
        b_n = torch.clamp(stock_price - 80, min=0).cuda() - torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'batafrei':
        b_n = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        c_n = b_n ** 2
    elif payoff == 'strangle_batafrei':
        strangle = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        batafrei = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        b_n = strangle + batafrei
        c_n = b_n ** 2
    elif payoff == 'put_bull':
        put = torch.clamp(80 - stock_price, min=0).cuda()
        bull = torch.clamp(stock_price - 80, min=0).cuda() - torch.clamp(stock_price - 120, min=0).cuda()
        b_n = put + bull
        c_n = b_n ** 2
    elif payoff == 'strangle_2batafrei':
        strangle = torch.clamp(80 - stock_price, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        batafrei = torch.clamp(stock_price - 80, min=0).cuda() - 2 * torch.clamp(stock_price - 100, min=0).cuda() + torch.clamp(stock_price - 120, min=0).cuda()
        b_n = strangle + 2 * batafrei
        c_n = b_n ** 2
    elif payoff == 'M_type':
        put=torch.clamp(80-stock_price, min=0).cuda()
        batafrei=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
        call=torch.clamp(stock_price-120, min=0).cuda()
        b_n=-put-2*batafrei-call+40
        c_n=b_n**2

    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0

In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ストラングル 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =np.arange(141,150,1)
strike_range=[200000]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #init_cost= BS_price(100,strike,0,0.3,1,'call')
    #init_cost= BS_price(100,80,0,0.3,1,'call')+BS_price(100,120,0,0.3,1,'call')-20
    #init_cost =BS_price(100,80,0,0.3,1,'call')-BS_price(100,120,0,0.3,1,'call')
    #init_cost =BS_price(100,80,0,0.3,1,'call')-2*BS_price(100,100,0,0.3,1,'call')+BS_price(100,120,0,0.3,1,'call')
    def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
      stock_price=np.array(S_0)
      for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

      return stock_price
    payoff='put_bull'
    if payoff == 'call':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    elif payoff == 'strangle':
      Call_T=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'bull':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'batafrei':
      Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff == 'strangle_batafrei':
      strangle=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=strangle+batafrei
    elif payoff == 'put_bull':
      put=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)
      bull=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=put+bull
    elif payoff == 'strangle_2batafrei':
      strangle=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=strangle+2*batafrei
    elif payoff == 'M_type':
      put=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)
      batafrei=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      call=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
      Call_T=-put-2*batafrei-call+40
    risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める


    for n in np.arange(maturity-1,-1,-1):
      Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
      stock_price=np.array(S_0)
      for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

      return stock_price

    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)


    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 2**6
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        if epoch % 1000==0:
            print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'/content/drive/MyDrive/MVH_NN/nonMarkov/hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'/content/drive/MyDrive/MVH_NN/nonMarkov/hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

0 6.724261884596444
1000 18.026580704065964
2000 18.02762104085849
3000 18.028923594923015
4000 18.02928164720055
5000 18.029331431586513
6000 18.028862744934543
7000 18.02909664061184
8000 18.029124502836794
9000 18.040437892544162
10000 18.04602262939619
11000 18.046021696135483
12000 18.04853786942357
13000 18.0485003821168
14000 18.048635257059004
15000 18.048629589078132
16000 18.04863461051258
17000 18.04864011727534
18000 18.048572683558348
19000 18.04864205360559
20000 18.04858915472431
21000 18.048657699767773
22000 18.04865954908098
23000 18.04866040155855
24000 18.04853453062816
25000 18.04866057110962
26000 18.048417660667212
27000 18.048658208800248
28000 18.048660061385533
29000 18.04864516446696
30000 18.04841598247424
31000 18.0484171888466
32000 18.04865786831533
33000 18.048660692907845
34000 17.953288057335953
35000 17.95329016759223
36000 17.953288187844407
37000 17.953226012888592
38000 17.95328771343293
39000 17.95328993907333
40000 17.95328999168629
41000 17.9532

,0,1,2,3,4,5,6
0,200000.0,33236.0,8.182589e-12,0.395727,18.048661,3762.0,0.0


In [ ]:
init_cost

array([26.1553924 , 20.22449372])

In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ブルスプレッド 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =np.arange(90,180,10)
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 2**8
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5*2

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

In [ ]:
#@title 関数 バタフライスプレッド 3

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 3 # @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    #b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0

In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた バタフライスプレッド 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[1000]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        -2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 1438.1684654860292,strike : 1000,loss : 78.28983347090555


,0,1,2,3,4,5,6
0,1000.0,99692.0,7.447706e-14,0.067658,78.289833,1438.0,7.105427e-15


In [ ]:
#@title 関数 ストラングル 4

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 4# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ストラングル 4
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[20]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2506.0909193619154,strike : 20,loss : 35.928793372686314


,0,1,2,3,4,5,6
0,20.0,99401.0,1.402683e-14,0.101557,35.928793,2506.0,1.463624e-08


In [ ]:
#@title 関数 ブルスプレッド 4

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 4# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    #b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ブルスプレッド 4
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[200]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        -np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2837.2500436999835,strike : 200,loss : 18.650189375033676


,0,1,2,3,4,5,6
0,200.0,73253.0,6.262376e-12,0.527502,18.650189,2837.0,0.0


In [ ]:
#@title 関数 バタフライスプレッド 4

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 4# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    #b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた バタフライスプレッド 4
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[2000]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        -2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params =np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25], [0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)

    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.0001) for net in nets]



    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2388.936821816009,strike : 2000,loss : 39.11137281954801


,0,1,2,3,4,5,6
0,2000.0,99999.0,9.582801e-11,0.001715,39.111373,2388.0,4.167922e-08


In [ ]:
#@title 関数 ストラングル 5

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 5# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ストラングル 5
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[30]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2417.4305912000127,strike : 30,loss : 22.967957356786627


,0,1,2,3,4,5,6
0,30.0,19926.0,1.036981e-08,0.129152,22.967957,2417.0,0.0


In [ ]:
#@title 関数 ブルスプレッド 5

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 5# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    #b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた ブルスプレッド 5
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[300]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        -np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        #+np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2322.4754613419645,strike : 300,loss : 10.248355588513107


,0,1,2,3,4,5,6
0,300.0,52718.0,1.009002e-11,0.546007,10.248356,2322.0,0.0


In [ ]:
#@title 関数 バタフライスプレッド 5

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch
torch.set_default_dtype(torch.double)
import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import random
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 5# @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 100  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])

    return stock_price

#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算

for n in np.arange(maturity-1,-1,-1):
    Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param


#1の位で切り上げ
init_cost=Call_T[0]


init_cost

#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算

def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
    stock_price=np.array(S_0)
    for _ in range(1,maturity+1):
        stock_price=stock_price.reshape(-1,1)*np.array([up,up*down,up*down**2])

    return stock_price
stock_price_T()
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob



#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    stock_price=torch.from_numpy(stock_price_T(maturity,branches,up,down,S_0)).cuda()
    a_n=torch.ones_like(stock_price).cuda()
    #コール
    #b_n=torch.clamp(stock_price-strike, min=0).cuda()
    #c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    #ストラングル
    #b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    #ブルスプレッド
    #b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
    #c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    #バタフライスプレッド
    b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
    c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1, 0,-1):
        prob_tensor=prob_list[n]

        Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
        Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
        Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
        Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
        Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)
        a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
        a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
        b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
        a_n = Cond_Exp_a - a_divide
        b_n= Cond_Exp_b - a_b_divide
        c_n= Cond_Exp_c - b_divide
        a_n=a_n.reshape(branches**(n-1),branches)
        b_n=b_n.reshape(branches**(n-1),branches)
        c_n=c_n.reshape(branches**(n-1),branches)

        stock_price=(stock_price[:,0]/up).reshape(branches**(n-1),branches)

    stock_price=stock_price[0]
    prob_tensor=prob_list[0]
    Cond_Exp_a=a_n.squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S=(a_n*(stock_price-S_0)).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_a_Delta_S_sq= (a_n*(stock_price-S_0)**2).squeeze(0)@ prob_tensor.squeeze(0)
    Cond_Exp_b= b_n.squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_b_Delta_S=(b_n*(stock_price-S_0)).squeeze(0)@prob_tensor.squeeze(0)
    Cond_Exp_c=c_n.squeeze(0)@ prob_tensor.squeeze(0)
    a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
    a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
    b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
    a_n = Cond_Exp_a - a_divide
    b_n= Cond_Exp_b - a_b_divide
    c_n= Cond_Exp_c - b_divide
    Hedge_Error=a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0=(Cond_Exp_b_Delta_S-Cond_Exp_a_Delta_S*init_cost)/Cond_Exp_a_Delta_S_sq
    return Hedge_Error,pi_0


In [ ]:
#期ごとのNN 2モデル, 多層化 これに決めた バタフライスプレッド 5
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[3000]
#nn.Moduleのサブクラスとして定義する
class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:
    def stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0)
        for _ in range(1,maturity+1):
            stock_price=stock_price.reshape(-1,1)*np.array([up_bin,up_bin*down_bin])
        return stock_price
    #コール
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    #ストラングル
    #Call_T=np.maximum(80-stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    #ブルスプレッド
    #Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        #-np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)\
    #バタフライスプレッド
    Call_T=np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)\
        -2*np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)\
        +np.maximum(stock_price_T_bin(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    for n in np.arange(maturity-1,-1,-1):
        Call_T=Call_T.reshape(branches_bin**(n),branches_bin)@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))#gain=np.sqrt(2)

    model_params = np.array([[0.07, 0.35], [0.03, 0.25]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 32
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", nn.ReLU())
            self.layers.add_module(f"st{0}", normalize)

            #3層
            self.layers.add_module(f"fc{1}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{1}", nn.ReLU())
            self.layers.add_module(f"st{1}", normalize)

            #4層
            self.layers.add_module(f"fc{2}", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu{2}", nn.ReLU())


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    nets = [NN().cuda() for _ in range(maturity)]
    for net in nets:
        net.apply(init_weights)



    optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in nets]


    num_epochs =10**5

    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):
        for optimizer in optimizers:
            optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, branches**(maturity-1), model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()

        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            lattice_points=torch.arange(0,3**n,1).reshape(-1,1).cuda()
            input_tensor=torch.concat([torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5,\
                                      (lattice_points),torch.sqrt(S*lattice_points),(S*lattice_points)**2],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:3**n]=nn.Softmax(dim=1)(nets[n](ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,1,1,1,0,0,0],[0,0,0,1,1,1,0,0,0]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(nets[0](ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor=torch.zeros(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, branches**(maturity-1), branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,branches**(maturity-1),1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:branches**n].reshape(branches**n,branches))


        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        #if epoch % 1000==0:
            #print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()



        for optimizer in optimizers:
            optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_t_x_non_Markov_3_custom_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_non_Markov.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df

処理時間 : 2304.1789774959907,strike : 3000,loss : 36.775618341753


,0,1,2,3,4,5,6
0,3000.0,68517.0,5.372656e-13,0.03106,36.775618,2304.0,0.0
